In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.regularizers import l2, l1
from tensorflow.keras import layers, Input
from tensorflow.keras import activations
from tensorflow.keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten, UpSampling2D
import matplotlib.pyplot as plt

TRAIN_BIG_CONV = False

## Run this first

In [ ]:
!pip install keras==2.7.0 tensorflow==2.7.0 scikit-image pillow

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
_ = tf.keras.datasets.fashion_mnist.load_data()
_ = tf.keras.datasets.cifar10.load_data()
_ = tf.keras.datasets.cifar100.load_data()
_ = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=True)

#### Different dataset this time

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
print(classes[y_train[0][0]])
plt.imshow(X_train[0])
plt.show()

### First run model from last meeting

In [ ]:
# kernel_regularizer = None
# bias_regularizer = None
kernel_regularizer = regularizers.l2(1e-5)
bias_regularizer = regularizers.l2(1e-5)
input_shape = X_train.shape[1:]

mlp_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = input_shape),
    tf.keras.layers.Dense(128, activation = 'relu',
                          kernel_regularizer=kernel_regularizer,
                          bias_regularizer=bias_regularizer),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=kernel_regularizer,
                          bias_regularizer=bias_regularizer),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(10, activation = 'softmax', kernel_regularizer=kernel_regularizer,
                          bias_regularizer=bias_regularizer)
])

In [ ]:
mlp_model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = mlp_model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_test, y_test), 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False)])

In [ ]:
input_shape = X_train.shape[1:]
classes_cnt = len(np.unique(y_train))
print(input_shape, X_train.shape, classes_cnt)
cnn_model = Sequential()
cnn_model.add(Conv2D(input_shape=input_shape, filters=4, kernel_size=(2, 2), padding='same', activation='relu'))
cnn_model.add(Conv2D(filters=4, kernel_size=(2, 2), padding='same', activation='relu'))
cnn_model.add(MaxPool2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(classes_cnt, activation='sigmoid'))

In [ ]:
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = cnn_model.fit(X_train, y_train, batch_size=128, epochs=5, validation_data=(X_test, y_test))

In [ ]:
cnn_model.count_params(), mlp_model.count_params()

In [ ]:
input_shape = X_train.shape[1:]
classes_cnt = len(np.unique(y_train))
print(input_shape, X_train.shape, classes_cnt)
cnn_model = Sequential()
cnn_model.add(Conv2D(input_shape=input_shape, filters=4, kernel_size=(2, 2), padding='same', activation='relu'))
cnn_model.add(Conv2D(filters=8, kernel_size=(2, 2), padding='same', activation='relu'))
cnn_model.add(MaxPool2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(filters=8, kernel_size=(2, 2), padding='same', activation='relu'))
cnn_model.add(Conv2D(filters=4, kernel_size=(2, 2), padding='same', activation='relu'))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dense(classes_cnt, activation='sigmoid'))

In [ ]:
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = cnn_model.fit(X_train, y_train, batch_size=128, epochs=5, validation_data=(X_test, y_test))

In [ ]:
cnn_model.count_params(), mlp_model.count_params()

## Bigger fish showcase

In [ ]:
big_fish = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=True)

In [ ]:
from PIL import Image
sample_image = np.asarray(Image.open('cat.img'))
sample_image.shape

In [ ]:
plt.imshow(sample_image)

In [ ]:
# ResNet accepts images of shape (224, 224, 3)
from skimage.transform import resize
image = resize(sample_image, (224, 224, 3))
plt.imshow(image)

In [ ]:
image.shape

In [ ]:
from keras.applications.resnet_v2 import decode_predictions
pred = big_fish.predict(image.reshape(1, *image.shape)) # this reshape is needed. First dim is the batch size
label = decode_predictions(pred)[0][0]
print(label[1], label[2])

### Quick tensorflow showcase

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data()
input_shape = X_train.shape[1:]
classes_cnt = len(np.unique(y_train))
print(input_shape, X_train.shape, classes_cnt)

In [ ]:
inputs = Input(shape=input_shape)
x = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(inputs)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Conv2D(filters=64, kernel_size=(2, 2), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Conv2D(filters=128, kernel_size=(2, 2), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Conv2D(filters=256, kernel_size=(2, 2), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x) 
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x) 
x = Dropout(0.3)(x)
outputs = Dense(classes_cnt, activation='sigmoid')(x) 
big_conv = Model(inputs=inputs, outputs=outputs)

In [ ]:
if TRAIN_BIG_CONV:
    history = big_conv.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test))
    big_conv.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
    big_conv.save_weights(os.path.join(os.getcwd(), 'big_conv.h5'))
else:
    big_conv.load_weights(os.path.join(os.getcwd(), 'big_conv.h5'))
    big_conv.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
big_conv.evaluate(X_test, y_test)

## Transfer Learning example

In [ ]:
(X_train_f, y_train_f), (X_test_f, y_test_f) = tf.keras.datasets.cifar10.load_data()
input_shape = X_train_f.shape[1:]
classes_cnt = len(np.unique(y_train_f))
print(input_shape, X_train.shape, classes_cnt)

In [ ]:
[[idx, layer] for idx, layer in enumerate(big_conv.layers)]

In [ ]:
last_layer, layers_cnt = 11, 14

In [ ]:
new_x = Dense(32, activation='relu')(big_conv.layers[last_layer].output)
new_x = Dropout(0.3)(new_x)
new_outputs = Dense(classes_cnt, activation='sigmoid')(new_x)
new_big_conv_net = Model(inputs=inputs, outputs=new_outputs)
new_big_conv_net.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
new_big_conv_net.evaluate(X_test_f, y_test_f)

In [ ]:
for idx in range(last_layer+1):
    new_big_conv_net.layers[idx].trainable = False

In [ ]:
big_conv.summary()

In [ ]:
new_big_conv_net.summary()

In [ ]:
new_big_conv_net.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = new_big_conv_net.fit(X_train_f, y_train_f, batch_size=32, epochs=1, validation_data=[X_test_f, y_test_f])

In [ ]:
big_conv.evaluate(X_test, y_test)

In [ ]:
new_big_conv_net.evaluate(X_test_f, y_test_f)

#### Let's change the last layers back to the one from big_conv

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data()
input_shape = X_train.shape[1:]
classes_cnt = len(np.unique(y_train))
print(input_shape, X_train.shape, classes_cnt)

In [ ]:
another_x = Dense(256, activation='relu')(new_big_conv_net.layers[last_layer].output)
another_x = Dropout(0.3)(another_x)
another_outputs = Dense(classes_cnt, activation='sigmoid')(another_x)
another_model = Model(inputs=inputs, outputs=another_outputs)
for idx in range(last_layer+1, layers_cnt+1):
    another_model.layers[idx].set_weights(big_conv.layers[idx].get_weights())
another_model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
another_model.evaluate(X_test, y_test)

In [ ]:
big_conv.evaluate(X_test, y_test)

In [ ]:
id(big_conv.layers[-1]) == id(another_model.layers[-1])

In [ ]:
(X_train_f, y_train_f), (X_test_f, y_test_f) = tf.keras.datasets.cifar10.load_data()
input_shape = X_train_f.shape[1:]
classes_cnt = len(np.unique(y_train_f))
print(input_shape, X_train.shape, classes_cnt)
test_inputs = Input(shape=input_shape)
test_x = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(test_inputs)
test_x = MaxPool2D(pool_size=(2, 2))(test_x)
test_x = Conv2D(filters=64, kernel_size=(2, 2), padding='same', activation='relu')(test_x)
test_x = MaxPool2D(pool_size=(2, 2))(test_x)
test_x = Conv2D(filters=128, kernel_size=(2, 2), padding='same', activation='relu')(test_x)
test_x = MaxPool2D(pool_size=(2, 2))(test_x)
test_x = Conv2D(filters=256, kernel_size=(2, 2), padding='same', activation='relu')(test_x)
test_x = MaxPool2D(pool_size=(2, 2))(test_x)
test_x = Flatten()(test_x)
test_x = Dense(1024, activation='relu')(test_x)
test_x = Dropout(0.3)(test_x)
test_x = Dense(32, activation='relu')(test_x)
test_x = Dropout(0.3)(test_x)
test_outputs = Dense(classes_cnt, activation='sigmoid')(test_x)
test_conv = Model(inputs=test_inputs, outputs=test_outputs)
test_conv.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
history = test_conv.fit(X_train_f, y_train_f, batch_size=32, epochs=1, validation_data=[X_test_f, y_test_f])

In [ ]:
test_conv.summary()

# Object Detection & segmentation showcase
[Follow this link](https://colab.research.google.com/github/tensorflow/tpu/blob/master/models/official/mask_rcnn/mask_rcnn_demo.ipynb)